# Most Popular model

This is base on this [notebook](https://www.kaggle.com/dvasyukova/expedia-hotel-recommendations/predict-hotel-type-with-pandas) by dune_dweller

Submission scored 0.30253, placing me 358 (about half)

## Initalise
set page width and load graphlab etc

In [1]:
# Set full page width
from IPython.core.display import HTML
HTML("""
<style>
.container {
    width: 100%;
}
</style>
""")

In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import datetime
import os as os

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

# for large number of cores in a machine
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)

2016-05-04 01:11:56,010 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1462324314.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


/home/ec2-user/anaconda2/envs/dato-env/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Create a summary

Sumamrise by srch_destination_id, tallying the bookings and click and creating a relevance score. 

In [5]:
train = gl.SFrame('Data/train_raw')
print "Train:", len(train)

Train: 37670293


In [6]:
summary = train.groupby(['srch_destination_id', 'hotel_cluster'], 
                        {'bookings':gl.aggregate.SUM('is_booking'), 
                         'clicks': gl.aggregate.COUNT()} )
summary

hotel_cluster,srch_destination_id,bookings,clicks
30,45970,4,23
63,43781,2,40
60,38280,0,1
25,31932,1,3
48,61467,0,4
82,23337,1,7
58,64858,0,1
89,17319,0,13
43,39387,0,10
47,35319,0,2


In [7]:
BOOKING_WEIGHT = 100
summary['clicks'] = summary['clicks'] - summary['bookings'] 
summary['relevance'] = summary['bookings']*BOOKING_WEIGHT + summary['clicks']
summary

hotel_cluster,srch_destination_id,bookings,clicks,relevance
30,45970,4,19,419
63,43781,2,38,238
60,38280,0,1,1
25,31932,1,2,102
48,61467,0,4,4
82,23337,1,6,106
58,64858,0,1,1
89,17319,0,13,13
43,39387,0,10,10
47,35319,0,2,2


# Find most popular hotel clusters by destination
Define a function to get most popular hotels for a destination group.
nlargest() function lets us save some time on sorting large groups.


In [8]:
def arg_max(d, k=3):
    topk = sorted(d.keys(), reverse=True)[:k]
    
    return  [str(d[k]) for k in topk]

In [29]:
most_popular = summary.groupby('srch_destination_id',
                       {'hotel_cluster': gl.aggregate.CONCAT('relevance', 'hotel_cluster')})

most_popular['hotel_cluster'] = most_popular['hotel_cluster'].apply(lambda d: arg_max(d, k=5))
most_popular.print_rows(5, max_column_width=40)

+---------------------+----------------------+
| srch_destination_id |    hotel_cluster     |
+---------------------+----------------------+
|        49249        | [43, 77, 48, 16, 40] |
|        51146        |         [99]         |
|        24527        | [82, 81, 53, 36, 62] |
|        30459        |       [18, 95]       |
|        15376        | [20, 38, 8, 78, 85]  |
+---------------------+----------------------+
[59455 rows x 2 columns]



In [32]:
test=gl.SFrame('Data/test_raw')

In [34]:
test = test.join(most_popular, how='left',on='srch_destination_id')
test

id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city
0,2015-09-03 17:09:54,2,3,66,174,37449
1,2015-09-24 17:38:35,2,3,66,174,37449
2,2015-06-07 15:53:02,2,3,66,142,17440
3,2015-09-14 14:49:10,2,3,66,258,34156
4,2015-07-17 09:32:04,2,3,66,467,36345
5,2015-07-21 11:58:45,2,3,66,311,48189
6,2015-07-29 07:58:39,2,3,66,311,48189
7,2015-08-01 20:13:15,2,3,66,348,24811
8,2015-11-07 12:29:09,2,3,66,311,48189
9,2015-11-08 16:21:37,2,3,66,311,48189


In [35]:
test['hotel_cluster'].num_missing()

14036

In [56]:
most_popular_all = summary.groupby('hotel_cluster',
                       {'relevance': gl.aggregate.SUM('relevance')})

most_popular_all = list(most_popular_all.topk('relevance', k=5)['hotel_cluster'].astype(str))
most_popular_all

['91', '48', '42', '59', '28']

In [59]:
test = test.fillna('hotel_cluster', most_popular_all)

In [62]:
test['hotel_cluster'] = test['hotel_cluster'].apply(lambda lst: ' '.join(lst))

In [63]:
test['srch_destination_id', 'hotel_cluster']

srch_destination_id,hotel_cluster
12243,55 37 5 22 11
14474,5
11353,0 31 77 91 59
8250,1 45 79 24 54
11812,91 2 42 59 48
11827,91 42 28 95 4
8271,95 2 21 98 33
8291,95 18 98 91 68
8250,1 45 79 24 54
9145,55 32 34 10 4


In [68]:
test.rename({'srch_destination_id': 'id'})
test['id', 'hotel_cluster'].save('Submissions/MostPopular.csv')
# submission scored 0.30253

In [ ]:
# Validation metric

In [69]:
# map@K is defined in this package.
import ml_metrics

ml_metrics.mapk(pred['hotel_cluster'], pred['hotel_cluster'], k=5)

1.0

In [105]:
#Grab 5% of the data for validation for use with mapk metric
rest, validate,  = train.random_split(0.95)

validate = validate.join(most_popular, how='left',on='srch_destination_id')
validate['srch_destination_id', 'hotel_cluster', 'hotel_cluster.1']


srch_destination_id,hotel_cluster,hotel_cluster.1
8803,5,"[46, 36, 12, 29, 81]"
8803,36,"[46, 36, 12, 29, 81]"
8803,36,"[46, 36, 12, 29, 81]"
12009,95,"[91, 77, 48, 59, 42]"
28494,95,"[77, 95, 33, 59, 91]"
8266,9,"[9, 95, 37, 68, 98]"
18488,8,"[8, 78, 99, 22, 95]"
18488,8,"[8, 78, 99, 22, 95]"
8746,82,"[8, 22, 99, 64, 9]"
8746,67,"[8, 22, 99, 64, 9]"


In [109]:
validate['hotel_cluster'].apply(lambda h: [h], list).dtype()


array.array

In [101]:
ml_metrics.mapk(validate['hotel_cluster'][:10], validate['hotel_cluster.1'][:10], k=5)

0.0

In [113]:
ml_metrics.apk([55], [56, 70, 98, 41, 55], k=5)

0.2